In [19]:
import pandas as pd
### use xarray for extracting temperature data from .nc files
import xarray as xr 
import numpy as np
import geopandas as gpd
import datetime
import os
import metpy.calc as mpcalc
from metpy.units import units
import rioxarray
import zipfile
import os

# Directory to extract files
extract_dir = '/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/nigeria/extracted'
gdf = gpd.read_file('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/adm0_shp/nigeria/clean_shp.shp')

In [20]:

# Ensure the directory exists
os.makedirs(extract_dir, exist_ok=True)


# Unzip the NetCDF files
for year in ['2017', '2018', '2019', '2020', '2021']:
    zip_path = f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/nigeria/unextracted/era5_land_dewpoint_{year}_11_12.netcdf.zip'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
                # Rename extracted files
        for file_name in zip_ref.namelist():
            if file_name.endswith('.nc'):
                old_file_path = os.path.join(extract_dir, file_name)
                new_file_path = os.path.join(extract_dir, f'era5_land_dewpoint_11_12_{year}.nc')
                os.rename(old_file_path, new_file_path)
           



In [24]:

# Ensure the directory exists
os.makedirs(extract_dir, exist_ok=True)


# Unzip the NetCDF files
for year in ['2018', '2019', '2020', '2021', '2022']:
    zip_path = f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/nigeria/unextracted/era5_land_dewpoint_{year}_01_04.netcdf.zip'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
                # Rename extracted files
        for file_name in zip_ref.namelist():
            if file_name.endswith('.nc'):
                old_file_path = os.path.join(extract_dir, file_name)
                new_file_path = os.path.join(extract_dir, f'era5_land_dewpoint_01_04_{year}.nc')
                os.rename(old_file_path, new_file_path)
           



In [25]:

# Ensure the directory exists
os.makedirs(extract_dir, exist_ok=True)


# Unzip the NetCDF files
for year in ['2018', '2019', '2020', '2021', '2022']:
    zip_path = f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/dewpoint/nigeria/unextracted/era5_land_dewpoint_{year}_01_04_xtra.netcdf.zip'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
                # Rename extracted files
        for file_name in zip_ref.namelist():
            if file_name.endswith('.nc'):
                old_file_path = os.path.join(extract_dir, file_name)
                new_file_path = os.path.join(extract_dir, f'era5_land_dewpoint_01_04_{year}_xtra.nc')
                os.rename(old_file_path, new_file_path)
           



In [31]:
data_arrays = []
for year in ['2017', '2018', '2019', '2020', '2021']:

    nc_file_11_12 = os.path.join(extract_dir, f'era5_land_dewpoint_11_12_{year}.nc')
    ds_11_12 = xr.open_dataset(nc_file_11_12)

    nc_file_01_04 = os.path.join(extract_dir, f'era5_land_dewpoint_01_04_{int(year ) + 1}.nc')
    ds_01_04 = xr.open_dataset(nc_file_01_04)

    nc_file_xtra= os.path.join(extract_dir, f'era5_land_dewpoint_01_04_{int(year ) + 1}_xtra.nc')
    ds_xtra = xr.open_dataset(nc_file_xtra)

    ds = xr.concat([ds_11_12, ds_01_04, ds_xtra], dim='valid_time')
    ds = ds.sortby('valid_time')

    # Rename 'valid_time' to 'time'
    ds = ds.rename({'valid_time': 'time'})
    
    # Convert time to the desired timezone (e.g., 'Brasilia')
    ds['time'] = ds['time'] + pd.Timedelta(hours=1)

    # Convert from Kelvin to Celsius
    ds['t2m'] = ds['t2m'] - 273.15
    ds['d2m'] = ds['d2m'] - 273.15

    # Calculate relative humidity
    ds['rh'] = mpcalc.relative_humidity_from_dewpoint(ds['t2m'] * units.degC, ds['d2m'] * units.degC)
    ds['rh'] = ds['rh'] * 100

    # Calculate THI using metpy's heat_index function
    ds['thi'] = mpcalc.heat_index(ds['t2m'] * units.degC, ds['rh'] * units.percent, mask_undefined=False)

    #get daily maximum of THI at each lat/lon position
    daily_max_thi = ds.resample(time='1D').max()

    data_arrays.append(daily_max_thi)

    print('appended')

    
concat_daily_max = xr.concat(data_arrays, dim='time')
#concat_daily_max.to_netcdf('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/nigeria/array/concat_array.nc')

appended
appended
appended
appended
appended


In [32]:
concat_daily_max

Magnitude,[[[23.780254364013672 23.585575103759766 23.480361938476562 ... 23.237537384033203 23.01985740661621 22.876314163208008] [24.499187469482422 24.356121063232422 24.28267478942871 ... 23.453933715820312 23.064085006713867 22.897443771362305] [25.29437828063965 25.167903900146484 25.1461124420166 ... 23.872365951538086 23.394901275634766 23.216064453125] ... [nan nan nan ... 97.48564910888672 97.56868743896484 97.55181884765625] [nan nan nan ... 97.21392059326172 97.27275085449219 97.2957763671875] [nan nan nan ... 97.34432983398438 97.424072265625 97.06207275390625]] [[26.765901565551758 26.366525650024414 26.538654327392578 ... 23.624908447265625 23.37706184387207 23.247583389282227] [26.786598205566406 26.54634666442871 26.8579044342041 ... 24.049516677856445 23.542421340942383 23.37847328186035] [27.081602096557617 27.008071899414062 27.43775177001953 ... 24.643619537353516 24.035125732421875 23.840627670288086] ... [nan nan nan ... 97.32805633544922 97.15267181396484 97.3653793334961] [nan nan nan ... 97.26901245117188 97.03373718261719 97.21952819824219] [nan nan nan ... 97.25236511230469 96.97250366210938 96.95404052734375]] [[23.994827270507812 23.512388229370117 23.25481414794922 ... 23.914783477783203 23.787572860717773 23.71453285217285] [24.364789962768555 24.11476707458496 24.01197052001953 ... 23.97720718383789 23.738351821899414 23.64300537109375] [24.84928321838379 24.6801815032959 24.901968002319336 ... 24.144906997680664 23.86284828186035 23.765607833862305] ... [nan nan nan ... 99.17637634277344 99.52210235595703 99.64469909667969] [nan nan nan ... 99.04475402832031 99.54576873779297 99.68346405029297] [nan nan nan ... 99.08565521240234 99.6772232055664 99.69227600097656]] ... [[20.98046112060547 19.646066665649414 19.11975860595703 ... 47.447914123535156 48.49505615234375 49.4986457824707] [21.1993465423584 21.105388641357422 20.514665603637695 ... 50.77015686035156 52.23957824707031 52.63041305541992] [21.510854721069336 21.788278579711914 21.379133224487305 ... 53.236263275146484 55.747474670410156 57.0380859375] ... [nan nan nan ... 95.05554962158203 94.92693328857422 94.93278503417969] [nan nan nan ... 94.69259643554688 94.71184539794922 94.70338439941406] [nan nan nan ... 94.70651245117188 94.798095703125 94.44933319091797]] [[24.858436584472656 24.65460968017578 24.925172805786133 ... 47.92437744140625 49.915626525878906 51.60594177246094] [26.305267333984375 25.815805435180664 26.10347557067871 ... 48.923797607421875 51.380149841308594 52.41847610473633] [28.038639068603516 27.291133880615234 26.86673927307129 ... 49.352237701416016 52.116172790527344 54.27696228027344] ... [nan nan nan ... 91.72740936279297 91.2922592163086 91.7744140625] [nan nan nan ... 90.81697082519531 90.65174102783203 90.45096588134766] [nan nan nan ... 89.80374908447266 89.50477600097656 89.19914245605469]] [[16.54169273376465 16.069934844970703 16.22947120666504 ... 21.449539184570312 18.657947540283203 16.72574806213379] [16.576875686645508 16.14708709716797 16.33545684814453 ... 29.05978012084961 27.625757217407227 24.726722717285156] [16.690998077392578 16.292560577392578 16.496402740478516 ... 35.974021911621094 35.48808288574219 33.94108200073242] ... [nan nan nan ... 88.59371185302734 88.50382232666016 88.91303253173828] [nan nan nan ... 88.09253692626953 88.29669189453125 88.61508178710938] [nan nan nan ... 88.77420043945312 89.64867401123047 89.54704284667969]]]
Units,dimensionless
Magnitude,[[[33.21663144561984 33.17941195434997 33.26396571265332 ... 32.38055298725766 32.42861900064685 32.464602622720975] [33.299399813016294 33.23764754666229 33.34800091054706 ... 32.62888044118887 32.67141580581671 32.68384830819241] [33.38200476434497 33.317396640777645 33.425273829036314 ... 32.84356756342788 32.89335889948745 32.86719706323413] ... [nan nan nan ... 32.3415713840061 32.294782002766965 32.401097615559934] [nan nan nan ... 32.305759853786924 32.441452874077754 32.55353291829431] [nan nan nan ... 32.193455166

In [33]:
concat_daily_max.to_netcdf('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/nigeria/array/concat_array.nc')

In [34]:
concat_daily_max = xr.open_dataset('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/nigeria/array/concat_array.nc')
concat_daily_max.rio.write_crs(4326, inplace=True)

<xarray.Dataset> Size: 215MB
Dimensions:      (latitude: 97, longitude: 121, time: 916)
Coordinates:
    number       int64 8B ...
  * latitude     (latitude) float64 776B 13.87 13.77 13.67 ... 4.472 4.372 4.272
  * longitude    (longitude) float64 968B 2.676 2.776 2.876 ... 14.58 14.68
  * time         (time) datetime64[ns] 7kB 2017-11-01 2017-11-02 ... 2022-05-02
    spatial_ref  int64 8B 0
Data variables:
    t2m          (time, latitude, longitude) float32 43MB ...
    d2m          (time, latitude, longitude) float32 43MB ...
    rh           (time, latitude, longitude) float32 43MB ...
    thi          (time, latitude, longitude) float64 86MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-20T09:54 GRIB to CDM+CF via cfgrib-0.9.1...

In [35]:
concat_daily_max['time'].max()

<xarray.DataArray 'time' ()> Size: 8B
array('2022-05-02T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    number       int64 8B ...
    spatial_ref  int64 8B 0

In [36]:
# Find all cells that intesect with each geometry
# get the mean of the maximum daily THI values across all lat/lons that intersect with the geometry
data_dic = {}

for idx, row in gdf.iterrows():

    adm2 = row['adm2']
    adm1 = row['adm1']
    # Clip the THI DataArray using the geometry
    try:
        clipped_thi = concat_daily_max.rio.clip([row.geometry], all_touched=True, crs=gdf.crs)
        
        # Convert the clipped THI DataArray to a DataFrame
        clipped_thi_df = clipped_thi.to_dataframe().reset_index()
        
        # In previous step, we get each cell's daily maximum THI value.
        # Here, we take the mean of the maximum daily THI values across all lat/lons that intersect with the geometry
        clipped_thi_df = clipped_thi_df.groupby('time').mean(numeric_only=True).reset_index()

        clipped_thi_df['adm2'] = adm2
        
        clipped_thi_df['adm1'] = adm1

        data_dic[f'{adm1}_{adm2}'] = clipped_thi_df

        print(f'{adm1}_{adm2} appended')

    except:
        continue

    


Abia_Aba North appended
Abia_Aba South appended
Borno_Abadam appended
Federal Capital Territory_Abaji appended
Akwa Ibom_Abak appended
Ebonyi_Abakaliki appended
Ogun_Abeokuta North appended
Ogun_Abeokuta South appended
Cross River_Abi appended
Imo_Aboh-Mbaise appended
Rivers_Abua-Odual appended
Federal Capital Territory_Abuja Municipal Area Council appended
Kogi_Adavi appended
Benue_Ado appended
Ogun_Ado-Odo-Ota appended
Ekiti_Ado-Ekiti appended
Oyo_Afijio appended
Ebonyi_Afikpo North appended
Ebonyi_Afikpo South appended
Niger_Agaie appended
Benue_Agatu appended
Lagos_Agege appended
Anambra_Aguata appended
Niger_Agwara appended
Imo_Ahiazu-Mbaise appended
Rivers_Ahoada East appended
Rivers_Ahoada West appended
Osun_Aiyedade appended
Osun_Aiyedire appended
Ekiti_Aiyekire (Gbonyin) appended
Kogi_Ajaokuta appended
Lagos_Ajeromi-Ifelodun appended
Kano_Ajingi appended
Cross River_Akamkpa appended
Oyo_Akinyele appended
Gombe_Akko appended
Edo_Akoko-Edo appended
Ondo_Akoko North East appended

In [37]:
final_concat = pd.concat(data_dic.values(), ignore_index=True)

In [38]:
final_concat.drop(columns=['latitude', 'longitude'], inplace=True)

In [41]:
final_concat.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/nigeria/daily_max_THI.csv')

In [42]:
df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/nigeria/daily_max_THI.csv')